# constrActive optimization

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
from eden.util import configure_logging
import logging
logger = logging.getLogger()
configure_logging(logger,verbosity=1)
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [2]:
from toolz import curry, pipe
from eden_chem.io.pubchem import download
from eden_chem.io.rdkitutils import sdf_to_nx

download_active = curry(download)(active=True)
download_inactive = curry(download)(active=False)

def get_pos_graphs(assay_id): return pipe(assay_id, download_active, sdf_to_nx, list)
def get_neg_graphs(assay_id): return pipe(assay_id, download_inactive, sdf_to_nx, list)

# Set up input graphs

In [3]:
from IPython.core.display import display
from eden_chem.display.rdkitutils import nx_to_image

def draw_mols(graphs, num=15, n_graphs_per_line=7):
    titles = [graph.graph.get('id',i) for i, graph in enumerate(graphs)]
    img = nx_to_image(graphs[:num], n_graphs_per_line=n_graphs_per_line, titles=titles)
    display(img)


def draw_rdkit(ideal_graphs, top_graphs, num=15, n_graphs_per_line=7):
    print('Ideal graphs: %d' % len(ideal_graphs))
    draw_mols(ideal_graphs, num=num, n_graphs_per_line=n_graphs_per_line)

    print('Top k graphs: %d' % len(top_graphs))
    draw_mols(top_graphs, num=num, n_graphs_per_line=n_graphs_per_line)

In [4]:
from eden.display import map_labels_to_colors
from eden.display import draw_graph_set       

class GraphDrawer(object):
    """GD."""

    def __init__(self, n_graphs_per_line=5, num_lines=4, size=8):
        """init."""
        self.n_graphs_per_line = n_graphs_per_line
        self.num_lines = num_lines
        self.size = size

    def fit(self, graphs):
        """fit."""
        label_colors = map_labels_to_colors(graphs)
        self.draw_params = dict(
            n_graphs_per_line=self.n_graphs_per_line, 
            size=self.size, 
            colormap='Paired', 
            vertex_color='_labels_', 
            vertex_color_dict=label_colors, 
            vertex_alpha=0.5, 
            edge_alpha=0.2)
        return self

    def draw(self, orig_graphs):
        graphs = orig_graphs[:self.num_lines*self.n_graphs_per_line]
        draw_graph_set(graphs, **self.draw_params)

/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/scipy/stats/_continuous_distns.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _stats
/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/utils/extmath.py:24: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/metrics/pairwise.py:30: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .pairwise_fast import _chi2_kernel_fast, _sparse_

# Experiment

In [5]:
from constrActive.utils import pre_process, _random_sample

def make_data(assay_id, max_size=20):
    # extract pos and neg graphs
    all_pos_graphs, all_neg_graphs = get_pos_graphs(assay_id), get_neg_graphs(assay_id)

    # remove too large and too small graphs and outliers
    args=dict(initial_max_size=4000, fraction_to_remove=.2, n_neighbors_for_outliers=3, remove_similar=True, max_size=max_size)
    logging.debug('\nPositive graphs')
    pos_graphs = pre_process(all_pos_graphs, **args)
    logging.debug('\nTest positive graphs')
    ts_pos_graphs = _random_sample([g for g in all_pos_graphs if g not in pos_graphs], len(pos_graphs)*2)

    logging.debug('\nNegative graphs')
    neg_graphs = pre_process(all_neg_graphs, **args)
    logging.debug('\nTest negative graphs')
    ts_neg_graphs = _random_sample([g for g in all_neg_graphs if g not in neg_graphs], len(neg_graphs)*2)
    logger.debug('-'*80)
    
    return pos_graphs, neg_graphs, ts_pos_graphs, ts_neg_graphs

/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/ipykernel_launcher.py:1: ImportWarning: Not importing directory '/home/fc334/Sync/Projects/constrActive': missing __init__.py
  """Entry point for launching an IPython kernel.
/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/neighbors/__init__.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .ball_tree import BallTree
/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/utils/random.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._random import sample_without_replacement
/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/utils/graph.py:15: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .graph_shortest_path import graph_shortest_path  # noqa
/home/fc334/miniconda2/envs/eden

In [6]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Perceptron
from constrActive.ideal_graph_estimator import IdealGraphEstimator

def make_ideal_graph_estimator(pos_graphs, neg_graphs, ts_pos_graphs, ts_neg_graphs, 
                               n_neighbors=50, n_landmarks=7, max_num_solutions=100):
    # estimate eprformance of EDeN classifier model
    est = EdenEstimator(r=3, d=6, penalty='perceptron', nbits=16, normalization=True, inner_normalization=True)
    ts_y = [1] * len(ts_pos_graphs) + [-1] * len(ts_neg_graphs)
    tr_y = [1] * len(pos_graphs) + [-1] * len(neg_graphs)
    tr_size = len(tr_y)
    scores = est.cross_val_score(pos_graphs+neg_graphs, tr_y, scoring='roc_auc', cv=10)
    tr_roc, tr_roc_std = np.mean(scores),np.std(scores)
    logger.info("CV   AUC ROC: %0.3f (+/- %0.3f) [%0.3f:%0.3f]" % (tr_roc, tr_roc_std, tr_roc-tr_roc_std, tr_roc+tr_roc_std))
    est.fit(pos_graphs+neg_graphs, tr_y)
    ts_preds = est.decision_function(ts_pos_graphs+ts_neg_graphs)
    ts_roc = roc_auc_score(ts_y, ts_preds)
    logger.info('Test AUC ROC: %.3f' % ts_roc)
    
    # make IGE 
    logger.info('-'*80)
    clf = Perceptron(n_iter=500) 
    ige = IdealGraphEstimator(  
        n_neighbors=n_neighbors,
        n_landmarks=n_landmarks,
        max_num_solutions=max_num_solutions)
    ige.fit(pos_graphs, neg_graphs)
    tr_xtr = ige.transform(pos_graphs+neg_graphs)
    n_graphs, n_feat = len(ige.ideal_graphs_), tr_xtr.shape[1]
    logger.info('Using %d selected constructs for %d features'%(n_graphs, n_feat))
    scores = cross_val_score(clf, tr_xtr, tr_y, cv=10, scoring='roc_auc')
    tr_ige_roc, tr_ige_roc_std = np.mean(scores),np.std(scores)
    logger.info('Train CV AUC ROC: %.3f +- %.3f [%0.3f:%0.3f]'%(tr_ige_roc, tr_ige_roc_std,tr_ige_roc-tr_ige_roc_std, tr_ige_roc+tr_ige_roc_std))
    clf.fit(tr_xtr, tr_y)
    ts_xtr = ige.transform(ts_pos_graphs+ts_neg_graphs)
    ts_ige_preds = clf.decision_function(ts_xtr)
    ts_ige_roc = roc_auc_score(ts_y, ts_ige_preds)
    logger.info('Test     AUC ROC: %.3f' % ts_ige_roc)
    return ige, est, (tr_roc, tr_roc_std, ts_roc, tr_ige_roc, tr_ige_roc_std, ts_ige_roc, n_graphs, n_feat, tr_size)

In [7]:
from eden.ml.estimator import EdenEstimator
from sklearn.metrics import roc_auc_score

def make_experiment(assay_id, max_size=20, n_neighbors=2, n_landmarks=50, max_num_solutions=10):
    def sort_graphs(orig_graphs, est):
        preds = est.decision_function(orig_graphs)
        graphs=[]
        for pred, graph in sorted(zip(preds, orig_graphs), reverse=True):
            graph.graph['id']='%.3f'%pred
            graphs.append(graph)
        return graphs

    pos_graphs, neg_graphs, ts_pos_graphs, ts_neg_graphs = make_data(assay_id, max_size=max_size)
    ige, est, rocs = make_ideal_graph_estimator(pos_graphs, neg_graphs, ts_pos_graphs, ts_neg_graphs, 
                                                n_neighbors=n_neighbors, n_landmarks=n_landmarks, max_num_solutions=max_num_solutions)
    ideal_graphs = sort_graphs(ige.ideal_graphs_, est)
    top_graphs = sort_graphs(pos_graphs, est)
    return rocs, ideal_graphs, top_graphs

def make_experiments(assay_ids, args):
    rocs_list = []
    for assay_id in assay_ids:
        print('\n\n')
        print('='*80)
        print('Exp: %s' % assay_id)
        print('_'*80)
        rocs, ideal_graphs, top_graphs = make_experiment(assay_id, **args)
        draw_rdkit(ideal_graphs, top_graphs, num=7*4, n_graphs_per_line=7)
        rocs_list.append(list(rocs))
    return rocs_list

In [8]:
import numpy as np
import pylab as plt
import time

def _extract_std(R):
    x = R[:,2]
    y1 = R[:,2]-R[:,1]
    y2 = R[:,2]+R[:,1] 
    xy12 = np.hstack([x.reshape(-1,1),y1.reshape(-1,1),y2.reshape(-1,1)])
    ids = np.argsort(x)
    xy12 = xy12[ids]
    x = np.array(xy12[:,0])
    y1 = np.array(xy12[:,1])
    y2 = np.array(xy12[:,2])
    return x, y1, y2

def plot(rocs_list, fname_prefix=''):
    def plot_(xx,yy,xlabel,ylabel,i, display_std=True):
        plt.subplot(1, n_graphs_per_line, i + 1)
        if display_std:
            plt.fill_between(x,y1,y2, color='b', alpha=0.2)
        plt.scatter(xx,yy)
        plt.plot([0.5, 1], [0.5, 1], color='gray', lw=1, linestyle='--')
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.grid(linestyle=":")


    R = np.array(rocs_list)
    x, y1, y2 = _extract_std(R)

    plot_size=5
    n_graphs_per_line=2
    plt.figure(figsize=(plot_size*n_graphs_per_line, plot_size))
        
    plot_(R[:,2],R[:,0],'ROC Test EDeN','ROC CV Train EDeN',0)
    plot_(R[:,2],R[:,5],'ROC Test EDeN','ROC Test IGE',1)
    
    timestamp = time.strftime('%Y_%m_%d_%H_%M_%S')
    plt.savefig('%s_roc_plots_%s.pdf'%(fname_prefix, timestamp),
                bbox_inches='tight',transparent=True,pad_inches=0)
    plt.show()

---

In [ ]:
#assay_ids=['492992','463230','588350','651610','624466','492952','463213','119','1224857','2326']
assay_ids=['492992','463230','588350','651610','624466','492952','463213']

args = dict(max_size=1000, n_neighbors=100, n_landmarks=7, max_num_solutions=100)

In [ ]:
%%time
rocs_list = make_experiments(assay_ids, args)
plot(rocs_list, 'exp2')




Exp: 492992
________________________________________________________________________________


/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


CV   AUC ROC: 0.786 (+/- 0.067) [0.719:0.853]
Test AUC ROC: 0.829
--------------------------------------------------------------------------------


/home/fc334/miniconda2/envs/eden_env/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


---